In [1]:
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
from tqdm import tqdm

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

# EDA number of labels

In [2]:
video_labels = pd.read_csv('train_labels.csv').fillna(0)
video_labels.head()

,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility
0,57583,82,Endzone,57583_000082_Endzone.mp4,1,H30,629,19,40,24,0.0,0,0.0,0.0
1,57583,82,Endzone,57583_000082_Endzone.mp4,1,V72,443,22,344,16,0.0,0,0.0,0.0
2,57583,82,Endzone,57583_000082_Endzone.mp4,1,V86,871,21,359,17,0.0,0,0.0,0.0
3,57583,82,Endzone,57583_000082_Endzone.mp4,1,V74,771,19,345,15,0.0,0,0.0,0.0
4,57583,82,Endzone,57583_000082_Endzone.mp4,1,V34,549,26,461,20,0.0,0,0.0,0.0


In [3]:
video_labels_with_impact = video_labels[video_labels['impact'] > 0]
videos = video_labels.video.unique()
num = []
for video in videos[1:]:
    v = video_labels[(video_labels["impact"]==1) & (video_labels["video"]==video)]
    print("number of labels: {} in video {}".format(len(v), video))
    num.append(len(v))

print("mean:{} max:{} min:{}".format(np.mean(num), np.max(num), np.min(num)))

number of labels: 18 in video 57583_000082_Sideline.mp4
number of labels: 16 in video 57584_000336_Endzone.mp4
number of labels: 16 in video 57584_000336_Sideline.mp4
number of labels: 20 in video 57584_002674_Endzone.mp4
number of labels: 23 in video 57584_002674_Sideline.mp4
number of labels: 19 in video 57586_000540_Endzone.mp4
number of labels: 21 in video 57586_000540_Sideline.mp4
number of labels: 28 in video 57586_001934_Endzone.mp4
number of labels: 27 in video 57586_001934_Sideline.mp4
number of labels: 10 in video 57586_004152_Endzone.mp4
number of labels: 11 in video 57586_004152_Sideline.mp4
number of labels: 19 in video 58106_002918_Endzone.mp4
number of labels: 19 in video 58106_002918_Sideline.mp4
number of labels: 20 in video 57594_000923_Endzone.mp4
number of labels: 18 in video 57594_000923_Sideline.mp4
number of labels: 26 in video 57596_002686_Endzone.mp4
number of labels: 24 in video 57596_002686_Sideline.mp4
number of labels: 22 in video 57597_000658_Endzone.mp4
n

# Make labels with all labels

In [4]:
video_labels = pd.read_csv('train_labels.csv').fillna(0)
# filter low-confident
video_labels_with_impact = video_labels[(video_labels['impact'] > 0)*(video_labels["confidence"]>1)*(video_labels["visibility"]>0)]

/home/ken/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/ken/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [5]:
video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str).str.zfill(3) + '.png'
video_labels['impact'] = video_labels['impact'].astype(int)+1
video_labels['x'] = video_labels['left']
video_labels['y'] = video_labels['top']
video_labels['w'] = video_labels['width']
video_labels['h'] = video_labels['height']

video_labels.to_csv("train_labels_all.csv")
video_labels.head()

,gameKey,playID,view,video,frame,label,left,width,top,height,impact,impactType,confidence,visibility,image_name,x,y,w,h
0,57583,82,Endzone,57583_000082_Endzone.mp4,1,H30,629,19,40,24,1,0,0.0,0.0,57583_000082_Endzone_001.png,629,40,19,24
1,57583,82,Endzone,57583_000082_Endzone.mp4,1,V72,443,22,344,16,1,0,0.0,0.0,57583_000082_Endzone_001.png,443,344,22,16
2,57583,82,Endzone,57583_000082_Endzone.mp4,1,V86,871,21,359,17,1,0,0.0,0.0,57583_000082_Endzone_001.png,871,359,21,17
3,57583,82,Endzone,57583_000082_Endzone.mp4,1,V74,771,19,345,15,1,0,0.0,0.0,57583_000082_Endzone_001.png,771,345,19,15
4,57583,82,Endzone,57583_000082_Endzone.mp4,1,V34,549,26,461,20,1,0,0.0,0.0,57583_000082_Endzone_001.png,549,461,26,20


# Generate impact only labels

In [ ]:
video_labels = pd.read_csv('train_labels.csv').fillna(0)
video_labels_with_impact = video_labels[(video_labels['impact'] > 0)*(video_labels["confidence"]>1)*(video_labels["visibility"]>0)]
for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
    frames = np.array([-4,-3,-2,-1,1,2,3,4])+row[1]
    video_labels.loc[(video_labels['video'] == row[0]) 
                                 & (video_labels['frame'].isin(frames))
                                 & (video_labels['label'] == row[2]), 'impact'] = 1
video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str).str.zfill(3) + '.png'
video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
video_labels['impact'] = video_labels['impact'].astype(int)+1
video_labels['x'] = video_labels['left']
video_labels['y'] = video_labels['top']
video_labels['w'] = video_labels['width']
video_labels['h'] = video_labels['height']
video_labels.head()
video_labels.to_csv("train_labels_4.csv")

/home/ken/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/home/ken/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
 31%|███       | 580/1887 [00:44<01:41, 12.83it/s]

In [ ]:
video_labels = pd.read_csv('train_labels.csv').fillna(0)
video_labels_with_impact = video_labels[(video_labels['impact'] > 0)*(video_labels["confidence"]>1)*(video_labels["visibility"]>0)]
for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
    frames = np.array([-3,-2,-1,1,2,3])+row[1]
    video_labels.loc[(video_labels['video'] == row[0]) 
                                 & (video_labels['frame'].isin(frames))
                                 & (video_labels['label'] == row[2]), 'impact'] = 1
video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str).str.zfill(3) + '.png'
video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
video_labels['impact'] = video_labels['impact'].astype(int)+1
video_labels['x'] = video_labels['left']
video_labels['y'] = video_labels['top']
video_labels['w'] = video_labels['width']
video_labels['h'] = video_labels['height']
video_labels.head()
video_labels.to_csv("train_labels_3.csv")

In [ ]:
video_labels = pd.read_csv('train_labels.csv').fillna(0)
video_labels_with_impact = video_labels[(video_labels['impact'] > 0)*(video_labels["confidence"]>1)*(video_labels["visibility"]>0)]
for row in tqdm(video_labels_with_impact[['video','frame','label']].values):
    frames = np.array([-2,-1,1,2,])+row[1]
    video_labels.loc[(video_labels['video'] == row[0]) 
                                 & (video_labels['frame'].isin(frames))
                                 & (video_labels['label'] == row[2]), 'impact'] = 1
video_labels['image_name'] = video_labels['video'].str.replace('.mp4', '') + '_' + video_labels['frame'].astype(str).str.zfill(3) + '.png'
video_labels = video_labels[video_labels.groupby('image_name')['impact'].transform("sum") > 0].reset_index(drop=True)
video_labels['impact'] = video_labels['impact'].astype(int)+1
video_labels['x'] = video_labels['left']
video_labels['y'] = video_labels['top']
video_labels['w'] = video_labels['width']
video_labels['h'] = video_labels['height']
video_labels.head()
video_labels.to_csv("train_labels_2.csv")

# Generate images from video

In [ ]:
def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
    video_path=f"{video_dir}/{video_name}"
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)
    if only_with_impact:
        boxes_all = video_labels.query("video == @video_name")
        print(video_path, boxes_all[boxes_all.impact > 1.0].shape[0])
    else:
        print(video_path)
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        frame += 1
        if only_with_impact:
            boxes = video_labels.query("video == @video_name and frame == @frame")
            boxes_with_impact = boxes[boxes.impact > 1.0]
            if boxes_with_impact.shape[0] == 0:
                continue
        img_name = f"{video_name}_frame{frame}"
        image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{str(frame).zfill(3)}.png')
        _ = cv2.imwrite(image_path, img)

In [ ]:
uniq_video = video_labels.video.unique()
video_dir = 'train'
out_dir = 'train_images'
!mkdir -p $out_dir
for video_name in uniq_video:
    mk_images(video_name, video_labels, video_dir, out_dir)

In [ ]:
uniq_video = video_labels.video.unique()
video_dir = 'train'
out_dir = 'train_images'
!mkdir -p $out_dir
for video_name in uniq_video:
    mk_images(video_name, video_labels, video_dir, out_dir, False)